# Ideal Data Structure But Poor Behavior

As we mentioned in the discussion, we design a version for ideal data structure. We implemented matrix multiplicaiton, vectorization and broadcast in this version. We can see the speed is super fast. However, the core idea that VI work is the Variational parameter $\phi$, $\gamma$ converge. (make the KL distance as small as possible to approximate the probability density) So if we break this rule, the algorithm cannot converge and even intractable. We can see with the same initialization, 10 times iteration, the time for this algorithm is only half of the vectorization version we adopt in the report. However, the mse is a factor of 0.1 to 0.01 in scale worse than the vectorization version. 

In [3]:
import numpy as np
from scipy.special import digamma, polygamma
import time

In [4]:
def simulation_data(M=500,k=10,V=1000,xi=40,gamma_shape=2,gamma_scale=1):
    """Simulation the data according to LDA process. Return a list 
    of N_d*V matrix(one-hot-coding) with length M."""
    
    res_list=[]   
    
    #hyperparameter
    alpha = np.random.gamma(shape=gamma_shape,scale=gamma_scale,size=k)
    BETA=np.random.dirichlet(np.ones(V),k)
    
    #document level
    N=np.random.poisson(lam = (xi-1),size = M) + 1 #avoid 0 words
    THETA=np.random.dirichlet(alpha,M)
    
    #word level
    for d in range(M):
        Z=np.random.multinomial(1,THETA[d,],N[d])
        Temp=Z@BETA
        W=np.zeros((N[d],V))
        for n in range(N[d]):
            W[n,]=np.random.multinomial(1, Temp[n,])
        res_list.append(W)
    return res_list, alpha, BETA

In [5]:
np.random.seed(123)
docs, alpha, BETA=simulation_data()

In [18]:
M=len(docs)
V=docs[0].shape[1]

In [19]:
docs_mat=np.zeros((M,V))
for d in range(M):
    docs_mat[d,]=docs[d].sum(axis=0)

In [20]:
docs_mat.shape

(500, 1000)

In [88]:
def EM_improve(docs,k, tol,max_iter=1000,initial_alpha_shape=5,initial_alpha_scale=2):
    """
    Latent Dirichlet Allocation: M-step.
    Do to a list of documnents. -- a list of matrix.
    -------------------------------------------------
    Input:
    docs: a list of one-hot-coding matrix ;
    k: a fixed positive integer indicate the number of topics.
    -------------------------------------------------
    Output:
    optimal Nd*k matrix Phi;
    optimal k*1 vector gamma;
    optimal k*V matrix BETA;
    optimal k*1 vector alpha.
    """
    
    #get basic iteration
    M,V=docs.shape
    N=docs.sum(axis=1)

    #initialization
    BETA0=np.random.dirichlet(np.ones(V),k)
    alpha0=np.random.gamma(shape=initial_alpha_shape,scale=initial_alpha_scale,size=k)
    GAMMA=np.array([alpha0+N[d]/k for d in range(M)])         
    
    BETA=BETA0
    alpha=alpha0
    alpha_dis = 1
    beta_dis = 1
    
    #relative tolerance: tolerance for each element
    tol=tol**2
    
    for iteration in range(max_iter):
        #update PHI,GAMMA
        Phi=(docs@BETA.T)*np.exp(digamma(GAMMA)-digamma(sum(GAMMA)))
        #Phi here is m*k matrix
        Phi=Phi/(Phi.sum(axis=1)[:,None]) #row sum to 1
        GAMMA=alpha[None,:]+Phi
        BETA_=np.zeros((k,V))
        for d in range(M): #documents
            BETA_ += BETA*np.exp(digamma(GAMMA[d,])-digamma(sum(GAMMA[d,])))[:,None]*docs[d,]
        BETA=BETA_/(BETA_.sum(axis=1)[:,None])   #rowsum=1
        #update alpha           
        z = M * polygamma(1, sum(alpha0))
        h=-M*polygamma(1,alpha0)
        g=M*(digamma(sum(alpha0))-digamma(alpha0))+(digamma(GAMMA)-digamma(GAMMA.sum(axis=1))[:,None]).sum(axis=0)
        c = (sum(g / h)) / (1/z + sum(1/h))
        alpha = alpha0 - (g - c)/h
        
        alpha_dis = np.mean((alpha - alpha0) ** 2)
        beta_dis = np.mean((BETA - BETA0) ** 2)
        alpha0 = alpha
        BETA0 = BETA
        if((alpha_dis <= tol) and (beta_dis <= tol)):
            break
          
    return alpha, BETA,Phi,GAMMA

In [113]:
%%time
a2,B2,phi,gamma=EM_improve(docs=docs_mat,k=10, tol=1e-3,max_iter=1000,initial_alpha_shape=2,initial_alpha_scale=1)

CPU times: user 25 s, sys: 173 ms, total: 25.2 s
Wall time: 24 s


In [114]:
def mmse(alpha,BETA,alpha_est,BETA_est):
    alpha_norm=alpha/np.sum(alpha)
    beta_mse=np.mean((BETA_est-BETA)**2)
    alpha_est_norm=alpha_est/np.sum(alpha_est)
    alpha_mse=np.mean((alpha_est_norm-alpha_norm)**2)
    return alpha_mse,beta_mse

In [115]:
mmse(alpha=alpha,BETA=BETA,alpha_est=a2,BETA_est=B2)

(0.01402539413832, 0.00099825513638563974)

In [116]:
%%time
a2,B2,phi,gamma=EM_improve(docs=docs_mat,k=10, tol=1e-3,max_iter=1000,initial_alpha_shape=100,initial_alpha_scale=0.01)

CPU times: user 24.2 s, sys: 124 ms, total: 24.3 s
Wall time: 23.1 s


In [117]:
mmse(alpha=alpha,BETA=BETA,alpha_est=a2,BETA_est=B2)

(0.021968082117301738, 0.00099825513638563974)